In [3]:
#imports
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split, KFold
import os
from os import listdir, path

In [4]:
paths = [
    "Resultados",
    "Data"
]
def crearPaths(paths=paths):
    for p in paths:    
        if not path.exists(f"{os.getcwd()}/{p}"):
            os.mkdir(f"{os.getcwd()}/{p}")
crearPaths()

In [7]:
#correr experimentos (ver como hay que modificar con lo de usar python y c++ juntos)
def correr_experimento(archivo_training, archivo_testing, archivo_salida, k, metodo, nitter, epsilon, alfa):
        
    # Crear proceso para ejecutar el codigo.
    if metodo == "kNN":
        process = subprocess.Popen(["./build/tp2", archivo_training, archivo_testing, archivo_salida, k, 0], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)
    else:
        process = subprocess.Popen(["./build/tp2", archivo_training, archivo_testing, archivo_salida, k, 1, nitter, epsilon, alfa], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar y leer salida de STDERR con el tiempo de ejecución.
    stdout, stderr = process.communicate() # communicate() devuelve una tupla (stdout, stderr)

    tiempo_de_ejecucion = str.splitlines(stderr)[0]
    #print(clog)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion con el metodo {'kNN' if metodo == 0 else 'PCA'} con los parámetros {archivo_training}, {archivo_testing}, k:{k} {', nitter:'+nitter if metodo == 1 else ''} {', epsilon:'+epsilon if metodo == 1 else ''} {', alfa:'+alfa if metodo == 1 else ''}.")
    return float(tiempo_de_ejecucion)


In [6]:
class Experimento:
    def __init__(self, trainingDataSet, testingDataSet, outputFile, metodo, vecinos, num_iter, alfa, epsilon):
        self.trainingDataset = trainingDataSet
        self.testingDataset = testingDataSet
        self.outputFile = outputFile
        self.metodo = metodo
        self.vecinos = vecinos
        self.num_iter = num_iter
        self.alfa = alfa
        self.epsilon = epsilon

    def print(self):
        print(f"Archivo De Entrenamiento: {self.trainingDataset}")
        print(f"Archivo De Prueba: {self.testingDataset}")
        print(f"Archivo De Salida: {self.outputFile}")
        print(f"Metodo: {self.metodo}")
        print(f"Vecinos: {self.vecinos}")
        print(f"Numero De Iteraciones: {self.num_iter}")
        print(f"Alfa: {self.alfa}")
        print(f"Epsilon: {self.epsilon}")

In [ ]:
#Ya no sirve
#experimentos=[Experimento("KNN", i, "no") for i in range(0, 1000)]

experimentos=[]

In [9]:
def splitDataset(testSize=0.3): #default 70-30 como el fernet
    dataset = pd.read_csv("train.csv")
    train, test = train_test_split(dataset, test_size=testSize);
    train.to_csv(path_or_buf="Data/random_train.csv", index=False)
    test.to_csv(path_or_buf="Data/random_validate.csv", index=False)
    return[train, test]

def k_foldDataSet(k):
    dataset = pd.read_csv("train.csv")
    kf = KFold(k, True)
    fold_indices = KFold.split(dataset)
    for key, indices in enumerate(fold_indices, 1):
        fold_dataset = dataset[index in indices] #no se si anda pero la idea es quedarse con los que estan dentro de lo indices del fold
        fold_dataset.to_csv(path_or_buf=f"Data/{k}_fold-{key}.csv", index=False)

    

In [10]:
train, test = splitDataset()
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
35660,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39011,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36078,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31964,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20941,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test.head()

In [ ]:
columnas=["SetEntrenamiento", "SetTesting", "Resultados", "Método", "Vecinos", "Numero de iteraciones", "Alfa", "Epsilon", "Tiempo"]
filas=[]
iteraciones = 5
for index, experimento in enumerate(experimentos):
    clear_output(wait=True)
    print(f"Experimento: {index} de {len(experimentos)} info:")
    experimento.print()
    
    tiempos = []
    for it in range(0, iteraciones):
        tiempo = correr_experimento(experimento.trainingDataSet, experimento.testingDataSet, experimento.outputFile, experimento.vecinos, experimento.metodo, experimento.num_iter, experimento.epsilon, experimento.alfa)
        tiempos.append(tiempo)
        
        
    media = np.median(tiempos)
    if experimento.metodo == "kNN":
        filas.append([experimento.trainingDataSet, experimento.testingDataSet, experimento.outputFile, experimento.metodo, experimento.vecinos, tiempo])
    else:
        filas.append([experimento.trainingDataSet, experimento.testingDataSet, experimento.outputFile, experimento.metodo, experimento.vecinos, experimento.num_iter, experimento.alfa, experimento.epsilon, tiempo])
    
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("./resultados/resultado.csv", index=False, header=True);
